In [1]:
import torch
torch.cuda.is_available()
print(torch.zeros(1).cuda())

import torch_geometric
import torch_geometric.nn as pyg_nn
from torch_sparse import SparseTensor, matmul
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import SAGEConv
print(torch_geometric.__version__)

import torch_scatter
import torch.nn as nn
import torch.nn.functional as F

import torch_geometric.utils as pyg_utils

from torch import Tensor
from typing import Union, Tuple, Optional
from torch_geometric.typing import (OptPairTensor, Adj, Size, NoneType, OptTensor)

from torch.nn import Parameter, Linear
from torch_sparse import SparseTensor, set_diag
from torch_geometric.nn.conv import MessagePassing
from torch_geometric.utils import remove_self_loops, add_self_loops, softmax

import networkx as nx
from deepsnap.hetero_graph import HeteroGraph
import copy
from copy import deepcopy
import pickle

import deepsnap
from deepsnap.graph import Graph
from deepsnap.batch import Batch
from deepsnap.dataset import GraphDataset
from deepsnap.hetero_gnn import forward_op
from deepsnap.hetero_graph import HeteroGraph
from sklearn.metrics import f1_score, roc_auc_score

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import json
from pathlib import Path as Data_Path
from os import listdir
from os.path import isfile, join
from itertools import combinations
from tqdm.notebook import tqdm

import graph_tool.all as gt
import json
print("graph-tool version: {}".format(gt.__version__.split(' ')[0]))

tensor([0.], device='cuda:0')
2.1.0
graph-tool version: 2.46


In [ ]:
import pickle

with open("./fixed_graph.pickle", "rb") as f:
    g_nx = pickle.load(f)

### N-hop neighbourhood

In [ ]:
pass

### Deepsnap

In [ ]:
# Create a DeepSNAP graph from NetworkX graph
ds_graph = Graph(g_nx)

In [ ]:
print("Number of vertices:", g_nx.number_of_nodes())
print("Number of edges:", g_nx.number_of_edges())

In [ ]:
task = 'link_pred'
dataset = GraphDataset([ds_graph], task=task, edge_train_mode='disjoint')

dataset_train, dataset_val, dataset_test = dataset.split(transductive=True, split_ratio=[0.8, 0.1, 0.1])

# dataset_train[0].to('cuda')
# dataset_val[0].to('cuda')
# dataset_test[0].to('cuda')

# dataset_train.to('cuda:0')
# dataset_val.to('cuda:0')
# dataset_test.to('cuda:0')

print(type(dataset_train))
print(dataset_train[0])
print(type(dataset_train[0]))

num_train_edges = dataset_train[0].edge_label_index.shape[1]
num_val_edges = dataset_val[0].edge_label_index.shape[1]
num_test_edges = dataset_test[0].edge_label_index.shape[1]

print("Train set has {} supervision (positive) edges".format(num_train_edges // 2))
print("Validation set has {} supervision (positive) edges".format(num_val_edges // 2))
print("Test set has {} supervision (positive) edges".format(num_test_edges // 2))

print("Train set has {} message passing edges".format(dataset_train[0].edge_index.shape[1]))
print("Validation set has {} message passing edges".format(dataset_val[0].edge_index.shape[1]))
print("Test set has {} message passing edges".format(dataset_test[0].edge_index.shape[1]))

print(dataset)

In [ ]:
# pickle.dump(dataset_train, open('./fixed_graphs/train.graph', 'wb'))

In [ ]:
# pickle.dump(dataset_val, open('./fixed_graphs/val.graph', 'wb'))

In [ ]:
# pickle.dump(dataset_test, open('./fixed_graphs/test.graph', 'wb'))

In [3]:
dataset_train = pickle.load(open('/home/asd27/scratch/fixed_graphs/train.graph', 'rb'))

In [4]:
dataset_val = pickle.load(open('/home/asd27/scratch/fixed_graphs/val.graph', 'rb'))

In [10]:
!pip install dill

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 2.4 MB/s eta 0:00:00 MB/s eta 0:00:01


In [13]:
import dill
with open('/home/asd27/scratch/fixed_graphs/test.graph', 'rb') as f:
    dataset_test = dill.load(f)

UnpicklingError: invalid load key, '\x00'.

In [12]:
# dataset_test = pickle.load(open('/home/asd27/scratch/fixed_graphs/test.graph', 'rb'))

In [ ]:
class LightGCNConv(MessagePassing):
    def __init__(self, in_channels, out_channels, normalize = True,
                 bias = False, **kwargs):  
        super(LightGCNConv, self).__init__(**kwargs)

        self.in_channels = in_channels
        self.out_channels = out_channels
        self.normalize = normalize

    def forward(self, x, edge_index, size = None):
        out = self.propagate(edge_index, x=(x, x))
        return out

    def message(self, x_j):
        out = x_j
        return out

    def aggregate(self, inputs, index, dim_size = None):
        node_dim = self.node_dim
        out = torch_scatter.scatter(inputs, index, dim=node_dim, reduce='mean')
        return out

In [ ]:
class LightGCN(torch.nn.Module):
    def __init__(self, train_data, num_layers, emb_size=16, initialize_with_words=False):
        super(LightGCN, self).__init__()
        self.convs = nn.ModuleList()
        assert (num_layers >= 1), 'Number of layers is not >=1'
        for l in range(num_layers):
            self.convs.append(LightGCNConv(input_dim, input_dim))

        # Initialize using custom embeddings if provided
        num_nodes = train_data.node_label_index.size()[0]
        self.embeddings = nn.Embedding(num_nodes, emb_size)
        if initialize_with_words:
            self.embeddings.weight.data.copy_(train_datanode_features)
        
        self.loss_fn = nn.BCELoss()
        self.num_layers = num_layers
        self.emb_size = emb_size
        self.num_modes = num_nodes

    def forward(self, data):
        edge_index, edge_label_index, node_label_index = data.edge_index, data.edge_label_index, data.node_label_index
        layer_embeddings = []
        
        x = self.embeddings(node_label_index)
        mean_layer = x

        # We take an average of ever layer's node embeddings
        for i in range(self.num_layers):
            x = self.convs[i](x, edge_index)
            # print("x shape",x.shape)
            # print("mean_layer shape",mean_layer.shape)
            mean_layer += x

        mean_layer /= 4

        # Prediction head is simply dot product
        nodes_first = torch.index_select(x, 0, edge_label_index[0,:].long())
        nodes_second = torch.index_select(x, 0, edge_label_index[1,:].long())

        # Since we don't want a rank output, we create a sigmoid of the dot product
        out = torch.sum(nodes_first * nodes_second, dim=-1) # FOR RANKING
        pred = torch.sigmoid(out)

        return torch.flatten(pred)

    def loss(self, pred, label):
        return self.loss_fn(pred, label)

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
print(type(dataset_train))

In [ ]:
args = {
    'device' : 'cuda' if torch.cuda.is_available() else 'cpu',
    'num_layers' : 3,
    'emb_size' : 32,
    'weight_decay': 1e-5,
    'lr': 0.04,
    'epochs': 600
}

datasets = {
    'train': dataset_train[0],
    'val': dataset_val[0],
    'test': dataset_test[0]
}
            
input_dim = datasets['train'].num_node_features
print(input_dim, args)

In [ ]:
# datasets['train'].to(args['device'])
# datasets['val'].to(args['device'])
# datasets['test'].to(args['device'])

In [ ]:
losses = []

def train(model, optimizer, args):
    val_max = 0
    best_model = model

    for epoch in range(1, args['epochs'] + 1):
        datasets['train'].to(args["device"])
        model.train()
        optimizer.zero_grad()
        pred = model(datasets['train'])
        loss = model.loss(pred, datasets['train'].edge_label.type(pred.dtype))

        loss.backward()
        optimizer.step()

        log = 'Epoch: {:03d}, Train: {:.4f}, Val: {:.4f}, Test: {:.4f}, Loss: {:.5f}, Val Loss: {:.5f}'
        score_train, train_loss = test(model, 'train', args)
        score_val, val_loss = test(model, 'val', args)
        score_test, test_loss = test(model, 'test', args)

        losses.append((train_loss, val_loss))

        print(log.format(epoch, score_train, score_val, score_test, train_loss, val_loss))
        if val_max < score_val:
            val_max = score_val
            best_model = copy.deepcopy(model)

    return best_model

def test(model, mode, args):
    model.eval()
    score = 0
    loss_score = 0

    data = datasets[mode]
    data.to(args["device"])

    pred = model(data)
    loss = model.loss(pred, data.edge_label.type(pred.dtype))
    score += roc_auc_score(data.edge_label.flatten().cpu().numpy(), pred.flatten().data.cpu().numpy())
    loss_score += loss.item()

    return score, loss_score

In [ ]:
torch.cuda.empty_cache()

In [ ]:
model = LightGCN(datasets['train'], args['num_layers'], emb_size=args['emb_size']).to(args['device'])
optimizer = torch.optim.Adam(model.parameters(), lr=args['lr'], weight_decay=args['weight_decay'])

best_model = train(model, optimizer, args)
log = "Train: {:.4f}, Val: {:.4f}, Test: {:.4f}, Train Loss: {:.5f}, Val Loss: {:.5f}, Test Loss: {:.5f}"
best_train_roc, train_loss = test(best_model, 'train', args)
best_val_roc, val_loss = test(best_model, 'val', args)
best_test_roc, test_loss = test(best_model, 'test', args)
print(log.format(best_train_roc, best_val_roc, best_test_roc, train_loss, val_loss, test_loss))

In [ ]:
#model_save
filename = 'best_lgcn.pkl'
torch.save(best_model, filename)

#datasets save
# with open('datasets_deepsnap_dict.pkl', 'wb') as f:
#     # pickle the dictionary and write it to the file
#     pickle.dump(datasets, f)

# model = torch.load('../lgcn.pkl')
# with open('../datasets_deepsnap_dict.pkl', 'rb') as f:
#     # load the pickled dictionary from the file
#     datasets = pickle.load(f)

In [ ]:
def plot_learning_curve(losses, title):
    train_loss, val_loss = zip(*losses)
    steps = list(range(1, len(train_loss) + 1))
    
    min_val_loss = np.round(np.min(val_loss), 3)
    # train_list = [math.log10(x) for x in train_loss]
    # val_list = [math.log10(x) for x in val_loss]
    
    plt.figure(figsize=(16, 6))
    plt.plot(steps, train_loss, '-r', label='Training Loss')
    plt.plot(steps, val_loss, '-b', label='Validation Loss')
    plt.hlines(min_val_loss, 1, 300, colors='k', linestyles='dotted', label='Min Validation Loss: {}'.format(min_val_loss))

    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    # plt.ylim((0.58, 0.71))
    plt.title(title)
    plt.legend(loc='upper right')
    plt.title(title)

    return plt

In [ ]:
plot_learning_curve(losses, 'test')

In [ ]:
data = datasets['test']
pred = model(data)

In [ ]:
pred

In [ ]:
len(pred)

In [ ]:
ds = datasets['test']
ds

In [ ]:
ds.edge_label

In [ ]:
type(ds.edge_label_index)

In [ ]:
dict1 = {'preds':pred.to('cpu').detach().numpy(), 'edge_label': ds.edge_label.to('cpu').detach().numpy(), 'src_node': ds.edge_label_index[0].to('cpu').detach().numpy(), 'dest_node': ds.edge_label_index[1].to('cpu').detach().numpy()}
df = pd.DataFrame(dict1)
df.head()

In [ ]:
df_0 = df[df['edge_label'] == 0]
df_0

In [ ]:
df_1 = df[df['edge_label'] == 1]
df_1

In [ ]:
df_lessthan = df_1[df_1['preds'] > 0.5]
df_lessthan

In [ ]:
df_lessthan.iloc[0]['src_node']

In [ ]:
g_nx[0]

In [ ]:
df_lessthan.iloc[0]["src_node"]
counter=0
neg=0
for x in g_nx.nodes:
    if(counter!=x):
        neg=neg+x-counter
        counter=x
    counter=counter+1
print(len(g_nx.nodes))
neg
        

In [ ]:
def row_op(row):

    if g_nx.nodes[row['src_node']]['name'] == g_nx.nodes[row['dest_node']]['name'] or g_nx.nodes[row['src_node']]['uri'] == g_nx.nodes[row['dest_node']]['uri']:
        return False
    return True

df_lessthan['is_valid'] = df_lessthan.apply(row_op,axis=1)
df_filter=df_lessthan[df_lessthan['is_valid']] 

In [ ]:
def getId(row):
    return pd.Series([g_nx.nodes[row['src_node']]['name'],g_nx.nodes[row['src_node']]['uri'],g_nx.nodes[row['src_node']]['type'],g_nx.nodes[row['dest_node']]['name'],g_nx.nodes[row['dest_node']]['uri'],g_nx.nodes[row['dest_node']]['type']])
    
df_filter[["src_name","src_uri","src_type","dest_name","dest_uri","dest_type"]]=df_filter[df_lessthan['is_valid']].apply(getId,axis=1)

In [ ]:
df_filter

In [ ]:
df_final = df_filter[(df_filter['src_name']=="Party mix") | ( df_filter['dest_name']=="Party mix") ]
df_final.sort_values('preds',ascending=False)
uri_list=list(df_final.apply(lambda x: x['src_uri'] if x['src_uri']!='' else x['dest_uri'], axis=1 ))

In [ ]:
# !pip install spotipy

In [ ]:
uri_list=[]
uri_list = list(df_final["dest_uri"])

In [ ]:
df_final

In [ ]:
uri_list

In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

def gettrackname(uri_list):
    # Replace the values below with your own Spotify API credentials
    client_id = 'd5566a60926740f3a8070889731a2d21'
    client_secret = 'eb5fc0638a1241c3a611186ff8d167e3'

    # Initialize the Spotify API client
    client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
    sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

    info = []
    for uri in uri_list:
        # Use the track method to get information about the track
        track_info = sp.track(uri)

        # Get the track name from the track information
        track_name = track_info['name']
        track_info = sp.track(uri)

        # Get the artist name from the track information
        artist_name = track_info['artists'][0]['name']
        info.append((track_name, artist_name))
        
    return info

print(gettrackname(uri_list))
